In [48]:
import csv
import time

from os import listdir
from os.path import expanduser, isfile, join

home = expanduser("~")
file_path = '%s/Downloads/' % home
bofa_prefix = 'ExportData'
et_prefix = 'pfmDownload'


EMA_LEN_SECTORS = 70
FIELDS_BOFA = ['symbol', 'description', 'quantity', 'unit cost', 'price', 'value',
               'cost basis']
NUMBER_OF_FIELDS_BOFA = len(FIELDS_BOFA)

FIELDS_ETRADE = ['symbol', 'last trade', 'change $', 'change %', 'day\'s gain', 'qty',
                 'price paid', 'total gain $', 'total gain %', 'market val']
NUMBER_OF_FIELDS_ETRADE = len(FIELDS_ETRADE)

                 
SECTORS = ['energy', 'staples', 'healthcare', 'Utilities', 'Materials', 'Technology', 'Industrial',
           'Finance', 'Consumer']
SYMBOLS = { 'energy':'XLE', 'staples':'XLP', 'healthcare':'XLV' }
            # , 'Utilities', 'Materials', 'Technology', 'Industrial',}
SPY_2X = 'SSO'



def summarize_positions(positions, fields):
    print('\nThere are %d different instruments invested. They are:' %len(positions))
    print(100*'-')

    qty = 'quantity'
    if not qty in fields:
        if 'qty' in fields:
            qty = 'qty'
        else:
            print('Dont know how to get quantity of a position')
            return
    cost_str = 'unit cost'
    if not cost_str in fields:
        if 'price paid' in fields:
            cost_str = 'price paid'
        else:
            print('Dont know how to get unit cost of a position')
            return

    for k in sorted(positions):
        pos = positions[k]
        shares = 0.0
        cost = 0.0
        for p in pos:
            shares += p[qty]
            cost += p[qty] * p[cost_str]
        print('%-26s : %9.2f shares    @ %9.2f  -- %s' % (k, shares, cost/shares,
                                                        pos[0]['description'] if 'description' in fields else ' '))
    print(100*'-')


def add_to_positions(positions, row, fields_index, fields):
    if not row or len(row) < len(fields) + 1:
        return
    symbol = row[fields_index['symbol']]
    if symbol[0].isdigit():
        return
    position = dict()
    for f in fields:
        value = row[fields_index[f]].strip().lower()
        if value.startswith('$'):
            value = value[1:]
        if value.endswith('%'):
            value = value[:-1]
        if not f == 'description':
            value = float(value.replace(',',''))
        position[f] = value
    if not symbol in positions:
        positions[symbol] = list()
    positions[symbol].append(position)
    

def getBofaPositions():
    bofa_files = [ f for f in listdir(file_path) if isfile(join(file_path, f)) and f.startswith(bofa_prefix) ]
    if not bofa_files or len(bofa_files) == 0:
        print('Could not find any BofA downloads')
        return None

    if len(bofa_files) > 1:
        bofa_files = sorted(bofa_files, reverse=True)
    bofa_file = bofa_files[0]
    bofa_date = bofa_file[len(bofa_prefix):-len('.csv')]
    bofa_date = '%s/%s/%s' % (bofa_date[2:4], bofa_date[:2], bofa_date[4:8])

    positions = dict()
    with open(join(file_path, bofa_file), 'rt') as f:
        csv_reader = csv.reader(f, delimiter=',')
        data_block_started = False
        for row in csv_reader:
            if row and len(row) > 0:
                if row[0].strip().lower() == "symbol":
                    header = [r.strip().lower() for r in row]
                    fields_index = dict()
                    for h in row:
                        if h.strip().lower() in FIELDS_BOFA:
                            fields_index[h.strip().lower()] = row.index(h)
                    data_block_started = True
                    continue
                if not data_block_started:
                    continue
                if row[0].strip().lower() == "balances":
                    break
                add_to_positions(positions, row, fields_index, FIELDS_BOFA[1:])
    return [bofa_date, positions]


def getEtradePositions():
    et_files = [ f for f in listdir(file_path) if isfile(join(file_path, f)) and f.startswith(et_prefix) ]
    if not et_files or len(et_files) == 0:
        print('Could not find any E*Trade downloads')
        return None

    if len(et_files) > 1:
        et_files = sorted(et_files, reverse=True)

    accounts = list()
    positions = dict()
    for et_file in et_files:
        with open(join(file_path, et_file), 'rt') as f:
            csv_reader = csv.reader(f, delimiter=',')
            data_block_started = False
            for row in csv_reader:
                if row and len(row) > 0:
                    if row[0].strip().lower().startswith('account:'):
                        account = row[0].strip().lower()
                        account = account[account.find(':')+1:]
                        if account in accounts:
                            print('Old download is being skipped: %s' % et_file)
                            break
                        accounts.append(account)
                    if row[0].strip().lower() == "symbol":
                        header = [r.strip().lower() for r in row]
                        fields_index = dict()
                        for h in row:
                            if h.strip().lower() in FIELDS_ETRADE:
                                fields_index[h.strip().lower()] = row.index(h)
                        data_block_started = True
                        continue
                    if not data_block_started:
                        continue
                    if row[0].strip().lower() == "cash":
                        break
                    add_to_positions(positions, row, fields_index, FIELDS_ETRADE[1:])
    return positions



today = time.strftime("%m/%d/%Y")
result = getBofaPositions()
bofa_date, bofa_positions = result[0], result[1]
if not bofa_date:
    print('As of today: %s there is no bofa downloaded data available!\n' % today)
else:
    print('As of today: %s we have bofa data from the date: %s\n' % (today, bofa_date))

summarize_positions(bofa_positions, FIELDS_BOFA)

et_positions = getEtradePositions()
summarize_positions(et_positions, FIELDS_ETRADE)



As of today: 08/10/2015 we have bofa data from the date: 08/08/2015


There are 13 different instruments invested. They are:
----------------------------------------------------------------------------------------------------
AAPL                       :    270.00 shares    @     69.37  -- apple inc
AGNC                       :    200.00 shares    @     32.22  -- american cap agy corp
BBEP                       :    400.00 shares    @     12.22  -- breitburn energy prtnrs
BIP                        :    100.00 shares    @     37.91  -- brookfield infrastucr pa
CIG                        :    737.00 shares    @      7.79  -- companhia energ de   adr
CMO                        :    250.00 shares    @     12.09  -- capstead mortgage cp new
DLR                        :     50.00 shares    @     66.00  -- digital rlty tr inc
GS                         :     50.00 shares    @    148.07  -- goldman sachs group inc
NLY                        :    200.00 shares    @     11.78  -- annaly cap mgm